In [1]:
import tensorflow as tf
import numpy as np
from google.colab import drive
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import LabelEncoder
import os
import tensorflow as tf
from tqdm import tqdm
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import Sequence
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Activation, Dropout, Flatten, Dense, Input, Layer
from tensorflow.keras.layers import Embedding, LSTM, add, Concatenate, Reshape, concatenate, Bidirectional
from tensorflow.keras.applications import VGG16, ResNet50, DenseNet201
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
from textwrap import wrap
from google.colab import drive
from tensorflow.keras.utils import plot_model

plt.rcParams['font.size'] = 12
sns.set_style("dark")
warnings.filterwarnings('ignore')

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [13]:
file_path = "/content/drive/MyDrive/projectMLDL/text/data_for_ml_clear.csv"
df = pd.read_csv(file_path)

In [14]:
np.unique(df["image"])

array(['190246_0.jpg', '190248_0.jpg', '190249.jpg', '192437_0.jpg',
       '192441_0.jpg', '192442_0.jpg', '192444_0.jpg', '199844_0.jpg',
       '199846_0.jpg', '199848_0.jpg', '199852_0.jpg', '200085_0.jpg',
       '200086_0.jpg', '200088_0.jpg', '200089_0.jpg', '200090_0.jpg',
       '200091_0.jpg', '200092_0.jpg', '200093_0.jpg', '200094_0.jpg',
       '200123_0.jpg', '200124_0.jpg', '200126_0.jpg', 'IMG_0225.JPG',
       'IMG_0226.JPG', 'IMG_0229.JPG', 'IMG_0230.JPG', 'IMG_0231.JPG',
       'IMG_0232.JPG', 'IMG_0233.JPG', 'IMG_0235.JPG', 'IMG_0257.JPG',
       'IMG_0258.JPG', 'IMG_0259.JPG', 'IMG_0386.JPG', 'IMG_0389.JPG',
       'IMG_0390.JPG', 'IMG_0391.JPG', 'IMG_20231007_141008.jpg',
       'IMG_25660913_083131.jpg', 'IMG_25660913_132501.jpg',
       'IMG_25660920_150148.jpg', 'S__12435466_0.jpg',
       'S__12435469_0.jpg', 'S__12435483_0.jpg', 'S__12435484_0.jpg',
       'S__12435487_0.jpg', 'S__12435491_0.jpg', 'S__23502851_0.jpg',
       'S__23502853_0.jpg', 'S__23502854_

In [15]:
df['image'].nunique()

67

In [16]:
images = df['image']
captions = df['caption']

In [18]:
all_captions = []

for caption in captions:
  caption_splited = ["" if word == "?" else word for word in caption.split()]
  all_captions.append(caption_splited)

In [19]:
len(all_captions)

335

In [20]:
all_captions[30:]

[['A', 'man', 'is', 'working', 'on', 'the', 'laptop'],
 ['A', 'man', 'is', 'sitting', 'in', 'the', 'room'],
 ['A', 'man', 'is', 'sitting', 'infrontof', 'the', 'laptop'],
 ['A', 'man', 'is', 'working', 'on', 'his', 'laptop'],
 ['A', 'student', 'is', 'sitting', 'in', 'the', 'room'],
 ['Three', 'men', 'are', 'wearing', '', 'a', 'thing'],
 ['A', 'man', '', 'wearing', '', 'a', 'shirt'],
 ['Six', 'people', 'are', 'standing', '', '', ''],
 ['A', 'woman', 'is', 'standing', '', '', ''],
 ['A', 'man', 'is', 'standing', '', '', ''],
 ['Three', 'men', 'are', 'standing', 'on', 'the', 'ground'],
 ['A', 'man', '', 'wearing', '', 'a', 'food'],
 ['A', 'man', '', 'wearing', '', 'a', 'shirt'],
 ['A', 'man', '', 'Wearing', '', 'a', 'thing'],
 ['A', 'man', '', 'wearing', '', 'a', 'coat'],
 ['A', 'man', 'is', 'sitting', 'on', 'the', 'chair'],
 ['A', 'man', '', 'wearing', '', 'a', 'shirt'],
 ['A', 'man', 'is', 'holding', '', 'a', 'thing'],
 ['A', 'man', '', 'wearing', '', 'a', 'glasses'],
 ['A', 'man', '', '

In [21]:
for i in all_captions :
  if len(i) != 7 :
    print(i)

In [22]:
# Assuming you have a DataFrame called unique_categories
categories = {
    'first_article': [],
    'subject': [],
    'verb_to_be': [],
    'verb': [],
    'preposition': [],
    'second_article': [],
    'objective': []
}

for caption in all_captions:
    for i, category in enumerate(categories.keys()):
        categories[category].append(caption[i].lower())

In [23]:
count = np.zeros(len(np.unique(categories['verb'])))

for i in categories['verb']:
  for idx, j in enumerate(np.unique(categories['verb'])):
    if i == j:
      count[idx] = count[idx] + 1

for i in range(len(np.unique(categories['verb']))):
  print(np.unique(categories['verb'])[i], ":", count[i])

eating : 12.0
holding : 27.0
playing : 10.0
sitting : 104.0
standing : 81.0
talking : 10.0
wearing : 78.0
working : 13.0
